## Summarizing speeches with NLP: I have a dream speech

Acording with International Data Corporation by 2025 we´ll be generatin 175 trillons gigabytes of digital data pro year. But most of this data-up to 95% will be unstructure, wich means it´s not organized into useful databases.

- In this notebook we will learn to use the NLTK to geneare a summary of one of the most famouse speeches of all time "I have a dream".
- After learn the basis we will use gesim to summarize speachs.
- Finnaly we will use a word cloud to produce a fun visual summary of the most fequently used words in a novel.

In data mining and ml there are two approaches to summarizing text: extraction and abstraction.

`Extraction-based:`  Uses various weighting functions to rank sentences by perceveived importance. Words uses more often are considered more important. Consequently, sentences containing those words are considred more important. The overall behavior is like using a yellow highligther to manually select keywords and sentences without altering the text.

This technique is good at pulling out important words and phrases.

`Abstraction:` Deeper compehension of the document to capture intent and produce more humal-like paraphasing. This includes creating completly new sentences.

Abstraction algorithms requred advanced and complicated deep learning methods and sophisticated language modeling.

#### Pipeline

   1. Point  web page containing the “I Have a Dream” speech.
   2. Loads the text as a string
   3. Tokenizes the text into words and sentences
   4. Removes stop words with no contextual content
   5. Counts the remaining words
   6. Uses the counts to rank the sentences 
   7. Displays the highest-ranking sentences


In [1]:
import re
import requests
import bs4
from collections import Counter
import nltk
from nltk.corpus import stopwords

**word_tokenize**

In [2]:
sentence = 'test - Hi this is a test about tokenize from nltk that is just a test.'
test_tokenize = nltk.word_tokenize(sentence)
print(test_tokenize)
print(list(sentence.split(' ')))

['test', '-', 'Hi', 'this', 'is', 'a', 'test', 'about', 'tokenize', 'from', 'nltk', 'that', 'is', 'just', 'a', 'test', '.']
['test', '-', 'Hi', 'this', 'is', 'a', 'test', 'about', 'tokenize', 'from', 'nltk', 'that', 'is', 'just', 'a', 'test.']


**Tokenize vs list(split(' '))**
- `Tokenize` performe better if there are errors in the sentences as 2 white spaces.(take care about comma, point..)
- Use `List` had the advantage that its alredy in python so we do no need to import anything (probably the cluster we are using doesn´t have that library install).

**sentence_tokenize**

In [3]:
speech_test = ''' Mr. Sherlock Holmes, who was usually very late in the mornings, save upon those not infrequent occasions when he was up all night, was seated at the breakfast table. I stood upon the hearth-rug and picked up the stick which our visitor had left behind him the night before. It was a fine, thick piece of wood, bulbous-headed, of the sort which is known as a “Penang lawyer.” Just under the head was a broad silver band nearly an inch across. “To James Mortimer, M.R.C.S., from his friends of the C.C.H.,” was engraved upon it, with the date “1884.” It was just such a stick as the old-fashioned family practitioner used to carry--dignified, solid, and reassuring.'''

In [4]:
sentences_test = nltk.sent_tokenize(speech_test)
for sentence in sentences_test:
    print(sentence)

 Mr. Sherlock Holmes, who was usually very late in the mornings, save upon those not infrequent occasions when he was up all night, was seated at the breakfast table.
I stood upon the hearth-rug and picked up the stick which our visitor had left behind him the night before.
It was a fine, thick piece of wood, bulbous-headed, of the sort which is known as a “Penang lawyer.” Just under the head was a broad silver band nearly an inch across.
“To James Mortimer, M.R.C.S., from his friends of the C.C.H.,” was engraved upon it, with the date “1884.” It was just such a stick as the old-fashioned family practitioner used to carry--dignified, solid, and reassuring.


**Frequency Diccionary**

In [5]:
frek_disc = nltk.FreqDist(nltk.word_tokenize(sentence.lower()))

In [6]:
frek_disc_ord = {k: v for k, v in sorted(frek_disc.items(), key=lambda item: item[1])}
for key, value in frek_disc_ord.items():
    print(key, value)

james 1
mortimer 1
m.r.c.s. 1
from 1
his 1
friends 1
of 1
c.c.h. 1
engraved 1
upon 1
with 1
date 1
1884. 1
just 1
such 1
a 1
stick 1
as 1
old-fashioned 1
family 1
practitioner 1
used 1
carry 1
-- 1
dignified 1
solid 1
and 1
reassuring 1
. 1
“ 2
to 2
” 2
was 2
it 2
the 3
, 6


**Use of \s+**

In [7]:
test_text = 'hola juanito    me gustaria invitarte a tomar una cerveza   cuando podrias venir?'
speech_2 = re.sub('\s+', ' ', test_text) 
print(speech_2)

hola juanito me gustaria invitarte a tomar una cerveza cuando podrias venir?


In [8]:
def remove_stop_words(speech_edit):
    """Remove stop words from string and return string."""
    stop_words = set(stopwords.words('english'))
    speech_edit_no_stop = ''
    for word in nltk.word_tokenize(speech_edit):
        if word.lower() not in stop_words:
            speech_edit_no_stop += word + ' '  
    return speech_edit_no_stop

In [9]:
def get_word_freq(speech_edit_no_stop):
    """Return a dictionary of word frequency in a string."""
    word_freq = nltk.FreqDist(nltk.word_tokenize(speech_edit_no_stop.lower()))
    return word_freq

In [10]:
def score_sentences(speech, word_freq, max_words):
    """Return dictionary of sentence scores based on word frequency."""
    sent_scores = dict()
    sentences = nltk.sent_tokenize(speech)
    for sent in sentences:
        sent_scores[sent] = 0
        words = nltk.word_tokenize(sent.lower())
        sent_word_count = len(words)
        if sent_word_count <= int(max_words):
            for word in words:
                if word in word_freq.keys():
                    sent_scores[sent] += word_freq[word]
            sent_scores[sent] = sent_scores[sent] / sent_word_count
    return sent_scores


In [11]:
url = 'http://www.analytictech.com/mb021/mlk.htm'
page = requests.get(url)
page.raise_for_status()
soup = bs4.BeautifulSoup(page.text, 'html.parser')
p_elems = [element.text for element in soup.find_all('p')]
speech = ' '.join(p_elems)

I just want to see the 10 most important phrases.

In [12]:
# Request input.
while True:
    max_words = input("Enter max words per sentence for summary: ")
    num_sents = input("Enter number of sentences for summary: ")
    if max_words.isdigit() and num_sents.isdigit():
        break
    else:
        print("\nInput must be in whole numbers.\n")

Enter max words per sentence for summary: 100
Enter number of sentences for summary: 10


In [13]:
# Fix typos, remove extra spaces, digits, and punctuation.
speech = speech.replace(')mowing', 'knowing')
speech = re.sub('\s+', ' ', speech) 
speech_edit = re.sub('[^a-zA-Z]', ' ', speech)
speech_edit = re.sub('\s+', ' ', speech_edit)

In [14]:
speech_edit_no_stop = remove_stop_words(speech_edit)
word_freq = get_word_freq(speech_edit_no_stop)
sent_scores = score_sentences(speech, word_freq, max_words)

In [15]:
# Print the top-ranked sentences.
counts = Counter(sent_scores)
summary = counts.most_common(int(num_sents))
print("\nSUMMARY:")
for i in summary:
    print(i[0])


SUMMARY:
From every mountainside, let freedom ring.
Let freedom ring from Lookout Mountain in Tennessee!
Let freedom ring from every hill and molehill in Mississippi.
Let freedom ring from the curvaceous slopes of California!
Let freedom ring from the snow capped Rockies of Colorado!
But one hundred years later the Negro is still not free.
From the mighty mountains of New York, let freedom ring.
From the prodigious hilltops of New Hampshire, let freedom ring.
And I say to you today my friends, let freedom ring.
I have a dream today.


**End.**